<a href="https://colab.research.google.com/github/Julian-Moncarz/Fractals/blob/main/California_Wild_Fire_Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For this project, I will anylize a dataset containing 100230 unique buildings in the California area and the wildfire's impact upon them. I will then train a model using PyTorch to predict the damage a given building has sustained based on the other variables in the dataset, such as zip code, year of construction and latatude and longitude. https://www.kaggle.com/code/devraai/california-wildfire-damage-prediction-analysis

In [37]:
# Import data
from google.colab import drive
drive.mount('/content/drive/')

# Convert CSV to pandas df and display it
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/Data/California_wildfire_data.csv')

df.head()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


<ipython-input-37-d9c3e2afd830>:7: DtypeWarning: Columns (13,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/My Drive/Data/California_wildfire_data.csv')


,_id,OBJECTID,* Damage,* Street Number,* Street Name,"* Street Type (e.g. road, drive, lane, etc.)","Street Suffix (e.g. apt. 23, blding C)",* City,State,Zip Code,...,Fire Name (Secondary),APN (parcel),Assessed Improved Value (parcel),Year Built (parcel),Site Address (parcel),GLOBALID,Latitude,Longitude,x,y
0,1,1,No Damage,8376.0,Quail Canyon,Road,NaN,Winters,CA,NaN,...,Quail,0101090290,510000.0,1997.0,8376 QUAIL CANYON RD VACAVILLE CA 95688,e1919a06-b4c6-476d-99e5-f0b45b070de8,38.474960,-122.044465,-1.358593e+07,4.646741e+06
1,2,2,Affected (1-9%),8402.0,Quail Canyon,Road,NaN,Winters,CA,NaN,...,Quail,0101090270,573052.0,1980.0,8402 QUAIL CANYON RD VACAVILLE CA 95688,b090eeb6-5b18-421e-9723-af7c9144587c,38.477442,-122.043252,-1.358579e+07,4.647094e+06
2,3,3,No Damage,8430.0,Quail Canyon,Road,NaN,Winters,CA,NaN,...,Quail,0101090310,350151.0,2004.0,8430 QUAIL CANYON RD VACAVILLE CA 95688,268da70b-753f-46aa-8fb1-327099337395,38.479358,-122.044585,-1.358594e+07,4.647366e+06
3,4,4,No Damage,3838.0,Putah Creek,Road,NaN,Winters,CA,NaN,...,Quail,0103010240,134880.0,1981.0,3838 PUTAH CREEK RD WINTERS CA 95694,64d4a278-5ee9-414a-8bf4-247c5b5c60f9,38.487313,-122.015115,-1.358266e+07,4.648497e+06
4,5,5,No Damage,3830.0,Putah Creek,Road,NaN,Winters,CA,NaN,...,Quail,0103010220,346648.0,1980.0,3830 PUTAH CREEK RD WINTERS CA 95694,1b44b214-01fd-4f06-b764-eb42a1ec93d7,38.485636,-122.016122,-1.358277e+07,4.648259e+06


**Step 1: Data Cleaning**

There a a bunch of useless columns and the dataset is huge, so we will delete three types of columns:

- Pure text (descriptions of damage eg)
- Duplicate info (id, unique identifier etc)
- Many many blank rows (Zip code)

In [34]:
df = df.drop(['OBJECTID',
              '_id',
              'Street Suffix (e.g. apt. 23, blding C)',
              'State',
              'Zip Code',
              'Community',
              'Battalion',
              'Incident Number (e.g. CAAEU 123456)',
              'Hazard Type',
              'Fire Name (Secondary)',
              'Distance - Residence to Utility/Misc Structure &gt; 120 SQFT',
              'APN (parcel)',
              'Distance - Propane Tank to Structure',
              'GLOBALID',
              'Site Address (parcel)',
              '# Units in Structure (if multi unit)',
              '* Structure Type'], axis=1)

KeyError: "['OBJECTID', '_id', 'Street Suffix (e.g. apt. 23, blding C)', 'State', 'Zip Code', 'Community', 'Battalion', 'Incident Number (e.g. CAAEU 123456)', 'Hazard Type', 'Fire Name (Secondary)', 'Distance - Residence to Utility/Misc Structure &gt; 120 SQFT', 'APN (parcel)', 'Distance - Propane Tank to Structure', 'GLOBALID', 'Site Address (parcel)', '# Units in Structure (if multi unit)', '* Structure Type'] not found in axis"

The goal is to train a **predictive** model which can determain the damage a structure will likely suffer before it is exposed to damage. Thus we will remove any data which could only be collected after a fire has occured.

In [35]:
df = df.drop(['* Incident Name',
              'Incident Start Date',
              'If Affected 1-9% - Where did fire start?',
              'If Affected 1-9% - What started fire?',
              '# of Damaged Outbuildings < 120 SQFT',
              '# of Non Damaged Outbuildings < 120 SQFT',
              ], axis=1)

In [36]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100230 entries, 0 to 100229
Data columns (total 24 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   * Damage                                      100230 non-null  object 
 1   * Street Number                               95810 non-null   float64
 2   * Street Name                                 94744 non-null   object 
 3   * Street Type (e.g. road, drive, lane, etc.)  87033 non-null   object 
 4   * City                                        68616 non-null   object 
 5   * CAL FIRE Unit                               100230 non-null  object 
 6   County                                        100200 non-null  object 
 7   Structure Defense Actions Taken               24470 non-null   object 
 8   Structure Category                            100230 non-null  object 
 9   * Roof Construction                           99

Data cleaning time!

Tasks:
